# Histogram fits with `pyhf`

Often we don't have a clear way to parametrize our fit templates, so we need to resort to MC simulations and use histograms as templates that we fit to data in the same bins.

We are going to use the [`pyhf`](https://github.com/scikit-hep/pyhf) package for these fits. The documentation can be found at https://pyhf.readthedocs.io/.

It can be installed with pip, e.g.

`pip install --user pyhf`

In addition we are using the `mplhep` package in this notebook to make histogram plotting more convenient and `iminuit` to extract uncertainties on fit parameters.

`pip install --user mplhep iminuit`

In [ ]:
import pyhf

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import mplhep as hep

Let's create 2 artificial histograms with 10 bins (having 11 bin boundaries). You could imagine these as two different background processes for which we have MC simulations on which we ran some event selection and created histograms for. For now, let's assume that the shape of these distributions comes out correctly and we only need to fit the normalization (for both templates independently) to data.

In [ ]:
bins = np.arange(11)

In [ ]:
hist1 = np.array([1.5, 3., 6., 7.5, 6.3, 6.6, 6., 4.5, 3. , 1.5])
hist2 = np.array([3. , 6., 9., 12., 15., 9. , 6., 3. , 0.3, 0.15])

In [ ]:
hep.histplot([hist1, hist2], bins)

We want to stack them since we think the sum of both should give us the expected data yield

In [ ]:
hep.histplot([hist1, hist2], bins, stack=True, histtype="fill")

Now, let's assume we observed the following data counts in each bin:

In [ ]:
data = np.array([ 4, 17, 26, 23, 34, 23, 21,  7,  8,  4])

In [ ]:
hep.histplot([hist1, hist2], bins, stack=True, histtype="fill")
hep.histplot(data, bins, histtype="errorbar", color="black")

Oftentimes, one plots errorbars, indicating 1 $\sigma$ confidence intervals on poisson distributed event counts to have some visualization on the expected spread. This can be done by passing the data as value for the `w2` argument of `histplot`.

<div class="alert alert-block alert-info">
    The parameter <code>w2</code> stands for the "sum of squares of weights" $\sum(w_i^2)$. The square root of this will give us an estimate for the standard deviaton.
    This is equal to the event counts for unweighted events (such as the observed data) and in case all values are integer <code>histplot</code> will plot an asymmetric interval based on the assumption of a poisson distribution.
</div>

In [ ]:
hep.histplot([hist1, hist2], bins, stack=True, histtype="fill")
hep.histplot(data, bins, histtype="errorbar", color="black", w2=data)

## One template fits it all

`pyhf` does fits using the Maximum-Likelihood method and uses the HistFactory ([CERN-OPEN-2012-016](https://cds.cern.ch/record/1456844)) template. In the simplemost case the pdf (probability density function) is just a product of poisson counts in each bin:

$$p(\vec n|\vec\lambda) = \prod_{\mathrm{bin}\, i} \mathrm{Pois}(n_i | \lambda_i)$$

where $\mathrm{Pois}(n_i | \lambda_i)$ is the Poisson distribution for $\lambda_i$ expected and $n_i$ observed counts. In our case $\lambda_i$ would be given by

$$\lambda_i = \mu_1 b_{i, 1} + \mu_2 b_{i, 2}$$

where $b_{i, 1}$ and $b_{i, 2}$ are the expected counts in bin $i$ of our 2 histograms and $\mu_1$ and $\mu_2$ are the normalization factors we want to fit. This pdf will define the Likelihood function that is later maximized to give the best fitting parameter values.

The general template is more complicated, allowing for constraint terms and separation into arbitrary channels - we will come back to that later.

Models in `pyhf` are defined with a json-like specification. That is, a nested structure of lists and dictionaries. The hierarchy is as follows:

* A model can have several **channels**. This can be used to e.g. define different normalization factors for different categories of selection (but they have to be orthogonal)
* Each channel can have several **samples**. Each sample comes with a histogram template.
* Each sample can have several **modifiers**. These will define the fit parameters. Modifiers can be free normalization factors or constraint parameters (more later)

In our case we can define a model with just one channel and two samples which each have a normalization factor (free parameter) as a modifier:

In [ ]:
samples = [
    {
        "name": "sample1",
        "data": list(hist1),
        "modifiers": [
            {"name": "mu1", "type": "normfactor", "data" : None}
        ],
    },
    {
        "name": "sample2",
        "data": list(hist2),
        "modifiers": [
            {"name": "mu2", "type": "normfactor", "data" : None}
        ],
    },
]
spec = {"channels" : [{"name" : "singlechannel", "samples" : samples}]}

# info: the `poi_name=None` is nescessary here since we don't want to do a hypothesis test
model = pyhf.Model(spec, poi_name=None)

<div class="alert alert-block alert-warning">
    The histogram bin contents need to be specified as a list (not a numpy array), such that we can really dump this into the text based json format.
</div>

Our specification would look like this as a json string (which can be simply stored in a text file):

In [ ]:
import json

In [ ]:
json.dumps(spec)

We will now run a *maximum likelihood fit* that gives us the parameters that maximize the likelihood (technically we will minimize the negative log-likelihood), the *maximum likelihood estimates* (mle).

In [ ]:
mu1, mu2 = pyhf.infer.mle.fit(data, model)

In [ ]:
mu1, mu2

We did not have to specify initial parameter values or bounds. For normalization factors the initial parameters are by default `1` and the bounds (fit range) is `[0, 10]`:

In [ ]:
model.config.suggested_init()

In [ ]:
model.config.suggested_bounds()

Let's look at the fitted templates, together with the data:

In [ ]:
hep.histplot([mu1 * hist1, mu2 * hist2], bins, stack=True, histtype="fill")
hep.histplot(data, bins, histtype="errorbar", color="black", w2=data)

<div class="alert alert-block alert-success">
    <b>Exercise:</b> Repeat the fit, but use the same normalization factor for both samples. <i>Hint:</i> parameters in the pyhf specification are just identified by their name and the same name can occur in multiple places in the spec.
</div>

## Uncertainties on fit parameters and the "post-fit" plot

Often, we are also interested in the uncertainties and correlations between fit parameters. We can use `iminuit` as a fitting backend for `pyhf` to extract them:

In [ ]:
pyhf.set_backend('numpy', 'minuit')

In [ ]:
parameters, correlations = pyhf.infer.mle.fit(data, model, return_uncertainties=True, return_correlations=True)

In [ ]:
parameters

In [ ]:
correlations

<div class="alert alert-block alert-success">
    <b>Questions</b>: Why are the fit parameters correlated? Why is the correlation coefficient negative?
</div>

We can visualize the impact of these uncertainties on our fit templates using [linear error propagation](https://en.wikipedia.org/wiki/Propagation_of_uncertainty).

In this simple case, let's calculate this manually:

$$\sigma_{\lambda_b}^2 = \left(\frac{\partial \lambda_b}{\partial \mu_1}\sigma_{\mu_1}\right)^2 + \left(\frac{\partial \lambda_b}{\partial \mu_2}\sigma_{\mu_2}\right)^2 + 2 \frac{\partial \lambda_b}{\partial \mu_1}\frac{\partial \lambda_b}{\partial \mu_2}\sigma_{\mu_1}\sigma_{\mu_2}\rho_{12} = \left(b_1\sigma_{\mu_1}\right)^2 + \left(b_2\sigma_{\mu_2}\right)^2 + 2 b_1 b_2 \sigma_{b_1}\sigma_{b_2}\rho_{12}$$

In [ ]:
sigma1, sigma2 = parameters[:, 1]
sigma1, sigma2

In [ ]:
hist_error = np.sqrt(
    (hist1 * sigma1) ** 2 + (hist2 * sigma2) ** 2 + 2 * hist1 * hist2 * sigma1 * sigma2 * correlations[0][1]
)
hist_error

For more generic templates/functions you can do that automatically. Either use the [`uncertainties`](https://pythonhosted.org/uncertainties/) package (for functions described by simple formulas) or calculate it numerically by varying each parameter up and down and using half of the resulting interval as a replacement for $\frac{\partial f}{\partial x_i}\sigma_{x_i}$.

A generic function for using this method with `pyhf` models is currently provided by the [`cabinetry`](https://github.com/alexander-held/cabinetry) package from Alexander Held with the function [`cabinetry.model_utils.calculate_stdev`](https://github.com/alexander-held/cabinetry/blob/c8668005e899556675b5e646e127908849bfe597/src/cabinetry/model_utils.py#L176)

In [ ]:
def errorband(bins, hist, hist_error, ax=None):
    ax = ax or plt.gca()
    n = hist
    s = hist_error

    def extend(x):
        return np.append(x, x[-1])

    ax.fill_between(
        bins,
        extend(n - s),
        extend(n + s),
        step="post",
        color="black",
        alpha=0.3,
        linewidth=0
    )

In [ ]:
hep.histplot([mu1 * hist1, mu2 * hist2], bins, stack=True, histtype="fill")
hep.histplot(data, bins, histtype="errorbar", color="black", w2=data)
errorband(bins, mu1 * hist1 + mu2 * hist2, hist_error)

This errorband is often referred to as "post-fit" uncertainties.

<div class="alert alert-block alert-info">
    <b>Note:</b> Since the same normalization factor is used across all bins, the uncertainty on the template depends on all data points. Therefore the interval is smaller than the expected fluctuation in each individual bin.
</div>

<div class="alert alert-block alert-success">
    <b>Question:</b> What would happen if you fit an independent template per histogram bin? What would the uncertainty look like?
</div>

<div class="alert alert-block alert-success">
    <b>Exercise [hard]:</b> Try it out! Hint: One way to do this is by defining a sample for each bin (with an individual normalization factor for each bin) and setting all other bins to 0.
</div>

# Advanced:  Uncertainties on the histogram templates

Our histogram templates are usually derived by MC simulations. We typically want to assign uncertainties on the templates themselves. Those can be of different origin, but mostly fall into the following categories:

* **MC stat. error**: Statistical uncertainty due to the limited simulated sample size. The relative uncertainty on the expected count in a histogram bin is given by $\sqrt{N}/N$ for a bin with $N$ simulated events or $\sqrt{\sum(w_i^2)}/{\sum w_i}$ for weighted events with weights $w_i$ in that bin.
* **Experimental uncertainties**: Uncertainties on detector simulation or reconstruction/calibration. Many of them are evaluated by re-running the full analysis chain with certain parameters varied up and down by one standard deviation of some measured/calibrated parameters.
* **Theory uncertainties**: Uncertainties on cross sections and on the choice of different theoretical models/approximations (e.g. parton shower) or parameters. Cross section uncertainties affect the normalization, while the others are typically evaluated by re-running the simulation with parameters changed or models/algorithms replaced.

In all cases we need to provide additional input to parametrize the Likelihood template. In `pyhf` this is done by specifying `modifiers` for the samples we want to assign uncertainties to. See

https://pyhf.readthedocs.io/en/v0.6.1/likelihood.html#modifiers

for an overview. Each modifier will come with additional parameters in the Likelihood template. We will discuss 2 cases:

* Uncorrelated uncertainties per bin
* Correlated uncertainty across all bins


## Uncorrelated uncertainties per bin

Uncorrelated in this sense means the uncertainty affects each bin individually. This is what we want for the **MC stat. error**.

Let's assume we want to fit just one histogram  this time (using `hist1` as a template) and we have determined the (absolute) uncertainty per bin as:

In [ ]:
hist1_uncorr_err = np.array([0.4 , 0.4 , 0.3 , 0.2 , 0.15, 0.4 , 0.45, 0.5 , 0.3 , 0.35])

In addition to our normalization factor, we add another modifier to the sample of type `shapesys`. This will create a template with an additional uncertainty for each bin. We need to specify the uncertainty per bin for the `"data"` entry of the modifier dictionary:

In [ ]:
samples = [
    {
        "name": "sample1",
        "data": list(hist1),
        "modifiers": [
            # first modifier: normalization factor
            {"name": "mu1", "type": "normfactor", "data" : None},
            # second modifier: uncorrelated uncertainties per bin
            {
                "name": "uncorrelated_uncertainties",
                "type": "shapesys",
                "data" : list(hist1_uncorr_err)
            }
        ],
    },
]
spec = {"channels" : [{"name" : "singlechannel", "samples" : samples}]}

In [ ]:
model_uncorr = pyhf.Model(spec, poi_name=None)

This will introduce one additional fit parameter per bin, so we have 11 parameters in total, 1 for the normalization factor and 10 for the uncorrelated uncertainty per bin.

In [ ]:
model_uncorr.config.npars

How does this work? Well, as mentioned these uncertainties usually come from the limited amount of MC samples. In some sense, this is an additional measurement with data from our MC simulation! So we build this into our model by adding **auxiliary data** that emulates this. The pdf for building the likelihood function becomes

$$p(\vec n, \vec a|\vec \lambda, \vec \gamma) = \prod_{\mathrm{data\,bin}\, i} \mathrm{Pois}(n_i | \lambda_i) \prod_{\mathrm{aux\,data\, bin}\, i} \mathrm{Pois}(a_i | \gamma_i a_i)$$

where in our example

$$\lambda_i = \mu_1 \gamma_i b_i$$

We determine the amount of auxiliary data $a_i$ per bin by asking "which effective number of entries would give us that relative uncertainty". That is given by

$$\frac{\sqrt{a_i}}{a_i} = \frac{\Delta b_i}{b_i} \rightarrow a_i = \left(\frac{b_i}{\Delta b_i}\right)^2$$

where $b$ is the expected count and $\Delta b$ the absolute uncertainty in the corresponding histogram bin.

So the auxiliary data for each bin is given by:

In [ ]:
(hist1 / hist1_uncorr_err) ** 2

which is also what `pyhf` calculated for us:

In [ ]:
model_uncorr.config.auxdata

The auxdata is set to match the expected counts exactly for the intial parameter values $\gamma_i = 1$:

In [ ]:
initial_parameters = model_uncorr.config.suggested_init()
hep.histplot(model_uncorr.expected_auxdata(initial_parameters), bins)
hep.histplot(model_uncorr.config.auxdata, bins, histtype="errorbar")
print(initial_parameters)

So the parameters $\gamma_i$ need to simultaneously fit the real and auxiliary data. They are therefore not completely free. We say they are "constrained".

<div class="alert alert-block alert-success">
    <b>Exercise</b> Try to manually tune the parameters in the following interactive plot. Observe how the templates that need to fit real and auxiliary data change. Also have a look at the value of the negative log likelihood (which is the objective we want to minimize in the fit). Why can't we get a perfect fit to the real data?
</div>

In [ ]:
import ipywidgets as widgets

In [ ]:
# You don't need to understand right now what is happening in this code cell, first focus on the application

sliders_gamma = {
    f"gamma{i}" : widgets.FloatSlider(
        1.0,
        min=0.1,
        max=2.0,
        orientation="vertical",
        continuous_update=False,
        description=f"γ{i}",
        layout=widgets.Layout(width='35px')
    )
    for i in range(1, 11)
}

slider_mu = widgets.FloatSlider(
    1.0, min=0.1, max=10.0, description="Norm factor", continuous_update=False
)

def plot(mu, **kwargs):
    fig, axs = plt.subplots(ncols=2, figsize=(20, 5))

    parameters = model_uncorr.config.suggested_init()
    
    parameters[0] = mu
    
    for k in kwargs:
        i = int(k.replace("gamma", ""))
        parameters[i] = kwargs[k]

    hep.histplot(model_uncorr.expected_actualdata(parameters), bins, ax=axs[0])
    hep.histplot(data, bins, yerr=np.sqrt(data), histtype="errorbar", color="black", ax=axs[0])
    hep.histplot(model_uncorr.expected_auxdata(parameters), bins, ax=axs[1])
    hep.histplot(
        model_uncorr.config.auxdata,
        bins,
        yerr=np.sqrt(model_uncorr.config.auxdata),
        histtype="errorbar", color="red",
        ax=axs[1]
    )
    axs[0].set_title("Actual data")
    axs[1].set_title("Auxiliary data")
    
    print(
        "Negative Log-Likelihood: "
        f"{- model_uncorr.logpdf(parameters, np.concatenate([data, model_uncorr.config.auxdata]))[0]:.3f}"
    )
    
interactive_plot = widgets.interactive_output(plot, dict(sliders_gamma, mu=slider_mu))
interactive_plot.layout.height = "350px"

def fit(b):
    parameters = pyhf.infer.mle.fit(np.concatenate([data, model_uncorr.config.auxdata]), model_uncorr)
    slider_mu.value = parameters[0]
    for k in sliders_gamma:
        i = int(k.replace("gamma", ""))
        sliders_gamma[k].value = parameters[i]
        
button = widgets.Button(description="Fit")
button.on_click(fit)

display(
    slider_mu,
    widgets.HBox([sliders_gamma[f"gamma{i}"] for i in range(1, 11)]),
    button,
    interactive_plot
)